## Task 2: Basic ML on MNIST and FashionMNIST

In [1]:
using MLDatasets, Plots, Measures, LaTeXStrings, StatsBase, Random

In [2]:

train_data = FashionMNIST.traindata(Float64);
train_imgs = train_data[1];
train_labels = train_data[2];

test_data = FashionMNIST.testdata(Float64);
test_imgs = test_data[1];
test_labels = test_data[2];
class_names = FashionMNIST.classnames()

┌ Warning: FashionMNIST.traindata() is deprecated, use `FashionMNIST(split=:train)[:]` instead.
└ @ MLDatasets C:\Users\joshu\.julia\packages\MLDatasets\soHS1\src\datasets\vision\fashion_mnist.jl:119
┌ Warning: FashionMNIST.testdata() is deprecated, use `FashionMNIST(split=:test)[:]` instead.
└ @ MLDatasets C:\Users\joshu\.julia\packages\MLDatasets\soHS1\src\datasets\vision\fashion_mnist.jl:127
┌ Warning: FashionMNIST.classnames() is deprecated, use `FashionMNIST().metadata["class_names"]` instead.
└ @ MLDatasets C:\Users\joshu\.julia\packages\MLDatasets\soHS1\src\datasets\vision\fashion_mnist.jl:138


10-element Vector{String}:
 "T-Shirt"
 "Trouser"
 "Pullover"
 "Dress"
 "Coat"
 "Sandal"
 "Shirt"
 "Sneaker"
 "Bag"
 "Ankle boot"

In [3]:
tshirt_imgs = train_imgs[:,:,train_labels .== 0];
trousers_imgs = train_imgs[:,:,train_labels .== 1];
pullover_imgs = train_imgs[:,:,train_labels .== 2];
dress_imgs = train_imgs[:,:,train_labels .== 3];
coat_imgs = train_imgs[:,:,train_labels .== 4];
sandle_imgs = train_imgs[:,:,train_labels .== 5];
shirt_imgs = train_imgs[:,:,train_labels .== 6];
sneaker_imgs = train_imgs[:,:,train_labels .== 7];
bag_imgs = train_imgs[:,:,train_labels .== 8];
ankle_boot_imgs = train_imgs[:,:,train_labels .== 9];

In [4]:
#only testing 1000 since my computer could plot the heatmap "oom" (out of memmory) only 8gb
tshirt_imgs_as_vectors = vcat([vec(tshirt_imgs[:,:,k])' for k in 1:1000]...);
trousers_imgs_as_vectors = vcat([vec(trousers_imgs[:,:,k])' for k in 1:1000]...);
dress_imgs_as_vectors = vcat([vec(pullover_imgs[:,:,k])' for k in 1:1000]...); #last(size(pullover_imgs))
coat_imgs_as_vectors = vcat([vec(dress_imgs[:,:,k])' for k in 1:1000]...); #last(size(dress_imgs))
sandle_imgs_as_vectors = vcat([vec(coat_imgs[:,:,k])' for k in 1:1000]...); #last(size(coat_imgs))
shirt_imgs_as_vectors = vcat([vec(sandle_imgs[:,:,k])' for k in 1:1000]...); #last(size(sandal_imgs))
sneaker_imgs_as_vectors = vcat([vec(shirt_imgs[:,:,k])' for k in 1:1000]...); #last(size(shirt_imgs))
bag_imgs_as_vectors = vcat([vec(bag_imgs[:,:,k])' for k in 1:1000]...); #last(size(bag_imgs))
ankle_boot_imgs_as_vectors = vcat([vec(ankle_boot_imgs[:,:,k])' for k in 1:1000]...); #last(size(ankle_boot_imgs))
train_data_all_classes = vcat(tshirt_imgs_as_vectors, trousers_imgs_as_vectors, dress_imgs_as_vectors, coat_imgs_as_vectors, sandle_imgs_as_vectors, shirt_imgs_as_vectors, sneaker_imgs_as_vectors, bag_imgs_as_vectors, ankle_boot_imgs_as_vectors);

In [5]:
@show size(train_data_all_classes)
#heatmap(train_data_all_classes,legend=false)

size(train_data_all_classes) = (9000, 784)


(9000, 784)

### Task 2.1 One vs. all (rest) Linear and Logistic

In [10]:
# firstly want to test if somthing is a tshirt or not.
train_labels_tshirt_classes = vcat(ones(1000),zeros(8000));
#shuffling data
Random.seed!(0)
perm = shuffle(1:9000)
train_data_all_classes = train_data_all_classes[perm,:];
train_labels_tshirt_classes = train_labels_tshirt_classes[perm,:];
#heatmap(train_data_all_classes,legend=false)

In [19]:
include("src/ML.jl")

LoadError: LoadError: MethodError: no method matching train_logistic()
[0mClosest candidates are:
[0m  train_logistic([91m::Any[39m, [91m::Any[39m, [91m::Any[39m; num_epochs, mini_batch_size, η) at c:\Users\joshu\Documents\MATH2504\Joshua-Crook-2504-2022-PROJECT3\src\ML.jl:17
in expression starting at c:\Users\joshu\Documents\MATH2504\Joshua-Crook-2504-2022-PROJECT3\src\ML.jl:52

### Task 2.2 One vs. One Linear and Logistic

### Task 2.3 Multi-class classifier (logistic softmax)

### Task 2.4 Comparison of results and discussion